# Experimentation

## Parser for OData Queries

In [141]:
# Test cases
test_cases = [
    "ColA eq 3",
    "ColB eq 'abc'",
    "ColA eq 3 and ColB eq 'abc'",
    "startswith(ColA, 'hello')",
    "(ColA eq 3 and ColB eq 'abc') or (ColC eq 55 and ColD = 'lel')",
    "startswith(ColA, 'hello') and substringof('hello', ColB)",
]

In [145]:
import re

def parse_odata_filter(query):
    # lt
    query = query.replace(' lt ', ' < ')
    # le
    query = query.replace(' le ', ' <= ')
    # gt
    query = query.replace(' gt ', ' > ')
    # ge
    query = query.replace(' ge ', ' >= ')
    # eq
    query = query.replace(' eq ', ' = ')
    # ne
    query = query.replace(' ne ', ' != ')
    # startswith(column, string)
    matches_sw = re.match('startswith\(.*?\)', query.lower())
    if matches_sw:
        span_sw = matches_sw.span()
        sw_query = query[span_sw[0]:span_sw[1]]
        # Extract text between brackets
        sw_terms = re.sub('.*\(', '', sw_query)
        sw_terms = re.sub('\).*', '', sw_terms)
        sw_terms = [s.strip() for s in sw_terms.split(',')]
        sw_terms[1] = re.sub('[^a-zA-Z0-9]', '', sw_terms[1])
        query = re.sub(sw_query.replace('(', '\(').replace(')', '\)'), f"{sw_terms[0]} LIKE '{sw_terms[1]}%'", query)
        
    # substringof(string, column)
    matches_so = re.search('substringof\(.*?\)', query.lower())
    if matches_so:
        span_so = matches_so.span()
        so_query = query[span_so[0]:span_so[1]]
        # Extract text between brackets
        so_terms = re.sub('.*\(', '', so_query)
        so_terms = re.sub('\).*', '', so_terms)
        so_terms = [s.strip() for s in so_terms.split(',')]
        so_terms[0] = re.sub('[^a-zA-Z0-9]', '', so_terms[0])
        query = re.sub(so_query.replace('(', '\(').replace(')', '\)'), f"{so_terms[1]} LIKE '%{so_terms[0]}%'", query)
    # day()
    # month()
    # year()
    # hour()
    # minute()
    # second()
    return query

In [146]:
parse_odata_filter(test_cases[-1])

"ColA LIKE 'hello%' and ColB LIKE '%hello%'"

In [147]:
for test_case in test_cases:
    print(parse_odata_filter(test_case))

ColA = 3
ColB = 'abc'
ColA = 3 and ColB = 'abc'
ColA LIKE 'hello%'
(ColA = 3 and ColB = 'abc') or (ColC = 55 and ColD = 'lel')
ColA LIKE 'hello%' and ColB LIKE '%hello%'


In [16]:
def parse_odata_filter(query, joins):
  # Replace lookup column with the associated table
  main_col_matches = re.findall('\w+\/', query)
  main_col_replacements = []
  for match in main_col_matches:
    main_col_replacements.append(joins[match[:-1]]['table'])
  for match, replacement in zip(main_col_matches, main_col_replacements):
    query = re.sub(match, replacement + '.', query)
  
  # Replace slashes with dots
  # query = query.replace('/', '.')
  # lt
  query = query.replace(' lt ', ' < ')
  # le
  query = query.replace(' le ', ' <= ')
  # gt
  query = query.replace(' gt ', ' > ')
  # ge
  query = query.replace(' ge ', ' >= ')
  # eq
  query = query.replace(' eq ', ' = ')
  # ne
  query = query.replace(' ne ', ' != ')
  # startswith(column, string)
  query = re.sub('startsWith', 'startswith', query, re.IGNORECASE)
  matches_sw = re.findall('startswith\(.*?\)', query)
  if len(matches_sw) > 0:
    for match in matches_sw:
      # Extract text between brackets
      sw_terms = re.sub('.*\(', '', match)
      sw_terms = re.sub('\).*', '', sw_terms)
      sw_terms = [s.strip() for s in sw_terms.split(',')]
      sw_terms[1] = re.sub('[^a-zA-Z0-9]', '', sw_terms[1])
      query = re.sub(match.replace('(', '\(').replace(')', '\)'), f"{sw_terms[0]} LIKE '{sw_terms[1]}%'", query)
      
      print(query)
      
  # substringof(string, column)
  matches_so = re.findall('substringof\(.*?\)', query, re.IGNORECASE)
  if len(matches_so) > 0:
    for match in matches_so:
      # Extract text between brackets
      so_terms = re.sub('.*\(', '', match)
      so_terms = re.sub('\).*', '', so_terms)
      so_terms = [s.strip() for s in so_terms.split(',')]
      so_terms[0] = re.sub('[^a-zA-Z0-9]', '', so_terms[0])
      query = re.sub(match.replace('(', '\(').replace(')', '\)'), f"{so_terms[1]} LIKE '%{so_terms[0]}%'", query)
  
  # day()

  # month()
  
  # year()
  
  # hour()
  
  # minute()
  
  # second()
  
  return query

In [20]:
query1 = {
    '$select': 'Id,tableTitle,parentDatasetID/datasetTitle,parentDatasetID/dataDomain,parentDatasetID/owner',
    '$filter': "startswith(parentDatasetID/dataDomain,'O') and startswith(parentDatasetID/owner,'B')",
    '$expand': 'parentDatasetID'
}

query2 = {
    '$select': 'Id,tableTitle,parentDatasetID/datasetTitle,parentDatasetID/dataDomain,parentDatasetID/owner',
    '$filter': "parentDatasetID/dataDomain eq 'Ops'",
    '$expand': 'parentDatasetID'
}

joins = {
    'parentDatasetID': {
        'table': 'dc_datasets'
    }
}

In [31]:
import re

def parse_odata_query(query):
    output = {
        'main_cols': [],
        'join_cols': [],
        'filter_query': '',
        'expand_cols': []
    }
    if not query:
        return
    for query, value in query.items():
        if query == '$filter':
            output['filter_query'] = value
        else:
            columns = [v.strip() for v in value.split(',')]
            if query == '$select':    
                for column in columns:
                    if '/' in column:
                        output['join_cols'].append(column)
                    else:
                        output['main_cols'].append(column)
            elif query == '$expand':
                output['expand_cols'].extend(columns)
    return output

In [32]:
parse_odata_query(query1)

{'main_cols': ['Id', 'tableTitle'],
 'join_cols': ['parentDatasetID/datasetTitle',
  'parentDatasetID/dataDomain',
  'parentDatasetID/owner'],
 'filter_query': "startswith(parentDatasetID/dataDomain,'O') and startswith(parentDatasetID/owner,'B')",
 'expand_cols': ['parentDatasetID']}

## Generate Fake Data
The fake data generator creates fake domains, datasets, and the full underlying tables.

- Domains: Arbitrary groups of datasets
- Datasets

The underlying tables include lookup tables, transactional tables, and metadata for all of them.

### 1. Lookup Tables
People: 1,000 people generated using `.profile()`:
- name
- sex
- address
- mail (email)
- birthdate
- company
- job

Job:
- Job: Choose 50 from people/job and randomise for all
- Job description: generate_key_thrust

Company:
- Choose 50 from people/company and randomise for all
- Company description: generate_key_thrust

### 2. Transactional Tables
Training:
- Date: date


### 3. Metadata
Domains: Ops, Manpower, Intel, Engineering, Training, Safety

Datasets:
- Title: Define list
- Use case: Key thrusts
- Owner: company
- Point of contact: first_name, last_name, company_email

Tables:
- Title: Define list, dependent on dataset
- Table description: Key thrusts
- Update frequency: daily, weekly, monthly, quarterly
- Site: url
- spId: md5 hash of title + description

Columns:
- Depends on menu

In [6]:
import numpy as np
import pandas as pd

from faker import Faker

In [7]:
fake = Faker()
Faker.seed(0)

In [31]:
def generate_key_thrust():
  verbs = ['exploit', 'enhance', 'establish', 'develop', 'grow', 'construct']
  return f"{fake.bs().capitalize()} to {np.random.choice(verbs)} {fake.catch_phrase().lower()}"

### Generate People Dataset

In [12]:
def get_person():
  fake_profile = fake.profile()
  output = {
    'name': fake_profile['name'],
    'sex': fake_profile['sex'],
    'address': fake_profile['address'],
    'mail': fake_profile['mail'],
    'birthdate': fake_profile['birthdate'],
    'company': fake_profile['company'],
    'job': fake_profile['job']
  }
  return output

In [61]:
people_df = []
for _ in range(1000):
  people_df.append(get_person())

people_df = pd.DataFrame(people_df).reset_index().rename(columns={'index': 'Id'})

# Get first 50 companies and proliferate to the rest
company_choices = people_df.company.iloc[:50].tolist()
people_df['company'] = np.random.choice(company_choices, 1000)

# Get first 50 jobs and proliferate to the rest
job_choices = people_df.job.iloc[:50].tolist()
people_df['job'] = np.random.choice(job_choices, 1000)

### Generate Jobs Lookup Table

In [62]:
jobs = people_df[['job']].drop_duplicates()
jobs['job_description'] = [generate_key_thrust() for _ in range(jobs.shape[0])]
jobs = jobs.reset_index(drop=True) \
  .reset_index() \
  .rename(columns={'index': 'Id'})

### Generate Company Lookup Table

In [63]:
companies = people_df[['company']].drop_duplicates()
companies['company_description'] = [generate_key_thrust() for _ in range(companies.shape[0])]
companies = companies \
  .reset_index(drop=True) \
  .reset_index() \
  .rename(columns={'index': 'Id'})

### Create Lookup in People Table

In [69]:
# Merge jobs
people_df = people_df \
  .merge(jobs[['job', 'Id']].rename(columns={'Id': 'jobId'}), left_on='job', right_on='job', how='left') \
  .drop('job', axis=1) \
  .rename(columns={'jobId': 'job'})

# Merge companies
people_df = people_df \
  .merge(companies[['company', 'Id']].rename(columns={'Id': 'companyId'}), left_on='company', right_on='company', how='left') \
  .drop('company', axis=1) \
  .rename(columns={'companyId': 'company'})

## Dataset with Missing Data

In [26]:
import numpy as np
import pandas as pd

from datetime import datetime
from faker import Faker
from sklearn.datasets import make_regression

fake = Faker()
Faker.seed(123)

### Generate Floats

In [147]:
def generate_cols(coltype, n_samples=200, n_cols=5, min_value=None, max_value=None,
                  missing_rates=[0.0, 0.0], ):
  valid_coltypes = ['float', 'int', 'cat', 'text', 'bool', 'datetime']
  assert coltype in valid_coltypes, "Select an appropriate coltype: {valid_coltypes}"
  assert n_cols > 0, f"You must generate at least one {coltype} column."
  assert missing_rates[0] <= missing_rates[1] and missing_rates[0] >= 0.0 and missing_rates[1] <= 1.0, "Set valid min and max missing rates."

  if coltype == 'float':
    data = [
      [fake.pyfloat(right_digits=2, min_value=min_value, max_value=max_value) for _ in range(n_cols)] for _ in range(n_samples)
    ]
  elif coltype == 'int':
    data = [
      [fake.randomize_nb_elements(100, min=min_value, max=max_value) for _ in range(n_cols)] for _ in range(n_samples)
    ]
  elif coltype == 'cat':
    data = [[fake.country_code() for _ in range(n_cols)] for _ in range(n_samples)]
  elif coltype == 'text':
    data = [[fake.sentence() for _ in range(n_cols)] for _ in range(n_samples)]
  elif coltype == 'bool':
    data = [np.random.choice([True, False], n_cols) for _ in range(n_samples)]
  elif coltype == 'datetime':
    data = [
      [fake.date_between(start_date=min_value, end_date=max_value) for _ in range(n_cols)] for _ in range(n_samples)
    ]
  
  # Convert to string
  data = pd.DataFrame(data, columns=[f"{coltype}_col{i+1}" for i in range(n_cols)])
  data = data.astype(str)

  # Add missing values
  col_missing_rates = np.random.uniform(low=missing_rates[0], high=missing_rates[1], size=n_cols)
  rands = np.random.random(size=data.shape)
  data[rands <= col_missing_rates] = ''

  # Add incorrect data types
  
  return data

In [148]:
# Generate a table
def generate_table(n_samples=200, n_float=1, n_int=1, n_cat=1, n_text=1, n_bool=1, n_dt=1,
                  min_missing=0.0, max_missing=0.0, float_min=-100, float_max=100,
                  int_min=None, int_max=None, start_date='-1y', end_date='today'):
  
  assert n_text > 0, "You must generate at least one text column."
  assert n_bool > 0, "You must generate at least one boolean column."
  assert n_dt > 0, "You must generate at least one datetime column."
  
  # Generate data
  data_float = generate_cols('float', n_samples=n_samples, n_cols=n_float,
                              min_value=float_min, max_value=float_max,
                              min_missing=min_missing, max_missing=max_missing)

  data_int = generate_cols('int', n_samples=n_samples, n_cols=n_int,
                            min_value=int_min, max_value=int_max,
                            min_missing=min_missing, max_missing=max_missing)

  data_cat = generate_cols('cat', n_samples=n_samples, n_cols=n_cat,
                            min_missing=min_missing, max_missing=max_missing)
  
  data_text = generate_cols('text', n_samples=n_samples, n_cols=n_text,
                            min_missing=min_missing, max_missing=max_missing)

  data_bool = generate_cols('bool', n_samples=n_samples, n_cols=n_bool,
                            min_missing=min_missing, max_missing=max_missing)

  data_dt = generate_cols('datetime', n_samples=n_samples, n_cols=n_dt,
                          min_value=start_date, max_value=end_date,
                          min_missing=min_missing, max_missing=max_missing)
  
  # Create dataframe
  df = pd.concat([data_float, data_int, data_cat, data_text, data_bool, data_dt], axis=1)
  df  = df.reset_index(drop=True) \
    .reset_index() \
    .rename(columns={'index': 'Id'})
  return df

In [149]:
d = generate_table(min_missing=0.2, max_missing=0.4)
d.apply(lambda x: x=='').mean()

float_col1       object
int_col1         object
cat_col1         object
text_col1        object
bool_col1        object
datetime_col1    object
dtype: object


Id               0.000
float_col1       0.235
int_col1         0.265
cat_col1         0.220
text_col1        0.295
bool_col1        0.275
datetime_col1    0.180
dtype: float64